In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:

import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')

In [3]:
site ='SiteA'
outfile = '/Users/ajc/AllPostProcessData/BarrowTransect/NGEE/WaterLevel-%s/'%site

In [11]:
sim_num = 4
save = False

if 'A' in site:
    infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_A/'

    simulation = 'Arun%sR22'%sim_num
    infile_simulated1 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun%sR21/'%sim_num
    infile_simulated2 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/%s/'%simulation
elif 'B' in site:
    infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_B/'

    simulation = 'Brun%sR22'%sim_num
    infile_simulated1 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteB/Brun%sR21/'%sim_num
    infile_simulated2 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteB/%s/'%simulation
elif 'C' in site:
    infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_C/'

    simulation = 'Crun%sR22'%sim_num
    infile_simulated1 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteC/Crun1R21/'
    infile_simulated2 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteC/%s/'%simulation

    


start_year = 2013
if start_year == 2013:
    if 'A' in site:
        dataset= 'NGEE_BRW_A_2012-09-09_2013-09-30_Soil_Pits.csv'
    elif 'B' in site:
        dataset= 'NGEE_BRW_B_2012-09-09_2013-09-30_Soil_Pits.csv'
    elif 'C' in site:
        dataset= 'NGEE_BRW_C_2012-09-09_2013-09-30_Soil_Pits.csv'
elif start_year == 2014:
    dataset = 'NGEE_BRW_A_2013-10-01_2014-09-30_Soil_Pits.csv'

infile_observed = infile + dataset

data = pd.read_csv(infile_observed, skiprows=5)
#print data.columns

DataObservedCenter = dict()
DataObservedRim = dict()

d = dict()
time_origin = datetime.datetime(start_year,1,1,0,0,0)
d['time'] = [pd.to_datetime(data['Timestamp'])]


Time = [(t - time_origin).dt.total_seconds()/86400 for t in d[u'time']]
Time = np.array([t for t in Time[0]])

start_time = np.where(Time == 0)[0]
print start_time
start_time = start_time[0]

for c in data.columns:
    
    if 'center' in c and 'VWC' in c:
        k = int(float(c[4:8])*100)
        print k
        d = np.array([x for x in data[c]])
        DataObservedCenter['VWC-%scm'%k] = d
    elif 'rim' in c and 'VWC' in c:
        k = int(float(c[4:8])*100)
        d = np.array([x for x in data[c]])
        print k, c
        DataObservedRim['VWC-%scm'%k] = d
        
        

#print DataObservedCenter['temperature-2cm'][0]
print DataObservedRim.keys()
print DataObservedCenter.keys()

[2735]
10 VWC_0.10m_pit2_rim
25 VWC_0.25m_pit2_rim
37 VWC_0.37m_pit2_rim
['VWC-25cm', 'VWC-10cm', 'VWC-37cm']
[]


In [12]:

if 'Arun' in simulation:
    depth_c = [] #['25cm','30cm', '42cm']
    depth_r = ['10cm','25cm', '37cm']
elif 'Brun' in simulation:
    depth_c = ['10cm','25cm', '48cm']
    depth_r = ['15cm','25cm', '42cm']
elif 'Crun' in simulation:
    depth_c = ['15cm','30cm', '42cm']
    depth_r = ['8cm','20cm', '34cm']

keys_c =[]
keys_r = []
DataSim1Center = dict()
DataSim1Rim = dict()
TimeSim1 = []

for d in depth_c:
    #print d
    file = infile_simulated1 + 'porosity-center-%s.dat'%d
    data_por = rd.ReadSingleFile(file)
    file = infile_simulated1 + 'sat-liq-center-%s.dat'%d
    data_sat = rd.ReadSingleFile(file)
    data = np.array([d1*d2 for d1,d2 in zip(data_sat['data'], data_por['data'])])
    DataSim1Center['VWC-%s'%d] = data
    keys_c.append('VWC-%s'%d)
for d in depth_r:
    file = infile_simulated1 + 'porosity-rim-%s.dat'%d
    print file
    data_por = rd.ReadSingleFile(file)
    file = infile_simulated1 + 'sat-liq-rim-%s.dat'%d
    data_sat = rd.ReadSingleFile(file)
    data = np.array([d1*d2 for d1,d2 in zip(data_sat['data'], data_por['data'])])
    DataSim1Rim['VWC-%s'%d] = data
    print data[0], data_sat['data'][0], data_por['data'][0]
    keys_r.append('VWC-%s'%d)
    
TimeSim1 = [t/86400. for t in data_sat['time']]
#print keys_c[0][-4:]

/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R21/porosity-rim-10cm.dat
0.09579173117655741 0.103013066394 0.929898842251
/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R21/porosity-rim-25cm.dat
0.08134736703268224 0.143228566216 0.567954907194
/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R21/porosity-rim-37cm.dat
0.08161846269228239 0.143836330659 0.567439827743


In [13]:


DataSim2Center = dict()
DataSim2Rim = dict()
TimeSim2 = []

for d in depth_c:
    file = infile_simulated2 + 'porosity-center-%s.dat'%d
    data_por = rd.ReadSingleFile(file)
    file = infile_simulated2 + 'sat-liq-center-%s.dat'%d
    data_sat = rd.ReadSingleFile(file)
    print file
    data = np.array([d1*d2 for d1,d2 in zip(data_sat['data'], data_por['data'])])
    #print data_por['data'][0], data_sat['data'][0]
    DataSim2Center['VWC-%s'%d] = data
print 'PASS'
for d in depth_r:
    file1 = infile_simulated2 + 'porosity-rim-%s.dat'%d
    data_por = rd.ReadSingleFile(file1)
    file2 = infile_simulated2 + 'sat-liq-rim-%s.dat'%d
    data_sat = rd.ReadSingleFile(file2)
    print file1, file2
    data = np.array([d1*d2 for d1,d2 in zip(data_sat['data'], data_por['data'])])
    #print data_por['data'][0], data_sat['data'][0]
    DataSim2Rim['VWC-%s'%d] = data

TimeSim2 = [t/86400. for t in data_sat['time']]
#print DataSim11Center['VWC-15cm'], len(DataSim11Center['VWC-15cm'])
#print TimeSim11
print 

PASS
/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/porosity-rim-10cm.dat /Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/sat-liq-rim-10cm.dat
/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/porosity-rim-25cm.dat /Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/sat-liq-rim-25cm.dat
/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/porosity-rim-37cm.dat /Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteA/Arun4R22/sat-liq-rim-37cm.dat



In [14]:
def put_axis(dep, loc=''):
    plt.xlabel('Day (%s)'%start_year,fontsize=12, fontweight='normal')
    plt.ylabel('Soil moisture (VWC) [-]',fontsize=12, fontweight='normal')
    plt.text(4, .9, 'Depth %s, %s, %s'%(dep,start_year, loc),fontsize=12, fontweight='normal')
    plt.ylim((0, 1.))
    plt.yticks(np.linspace(0,1., 5))
    plt.xlim((0, 280))
    plt.xticks(np.linspace(0,280, 5))
    plt.legend(loc='center left')
    #plt.axhline(y=273.15,color='b')


In [9]:
# Polygon Center
fig, axs = plt.subplots(1,3, figsize=(10,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()
#print DataSim11Center['VWC-15cm'], len(DataSim11Center['VWC-15cm'])
plt.subplot(131)
plt.plot(Time[start_time:], DataObservedCenter[keys_c[0]][start_time:],'r', label='Observed')
plt.plot(TimeSim1, DataSim1Center[keys_c[0]],'k')
plt.plot(TimeSim2, DataSim2Center[keys_c[0]],'k', label='Simulated')
put_axis(keys_c[0][-4:], 'center')

plt.subplot(132)
plt.plot(Time[start_time:], DataObservedCenter[keys_c[1]][start_time:],'r', label='Observed')
plt.plot(TimeSim1, DataSim1Center[keys_c[1]],'k')
plt.plot(TimeSim2, DataSim2Center[keys_c[1]],'k',label='Simulated')
put_axis(keys_c[1][-4:], 'center')

plt.subplot(133)
plt.plot(Time[start_time:], DataObservedCenter[keys_c[2]][start_time:],'r', label='Observed')
plt.plot(TimeSim1, DataSim1Center[keys_c[2]],'k')
plt.plot(TimeSim2, DataSim2Center[keys_c[2]],'k',label='Simulated')
put_axis(keys_c[2][-4:], 'center')

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'vwc-center%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

IndexError: list index out of range

In [15]:

# Polygon Rim
fig, axs = plt.subplots(1,3, figsize=(10,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()
#print DataObservedRim.keys()
plt.subplot(131)
plt.plot(Time[start_time:], DataObservedRim[keys_r[0]][start_time:],'r',label='Observed')
plt.plot(TimeSim1, DataSim1Rim[keys_r[0]],'k')
plt.plot(TimeSim2, DataSim2Rim[keys_r[0]],'k',label='Simulated')
put_axis(keys_r[0][-4:],'Rim')

plt.subplot(132)
plt.plot(Time[start_time:], DataObservedRim[keys_r[1]][start_time:],'r',label='Observed')
plt.plot(TimeSim1, DataSim1Rim[keys_r[1]],'k')
plt.plot(TimeSim2, DataSim2Rim[keys_r[1]],'k',label='Simulated')
put_axis(keys_r[1][-4:],'Rim')

plt.subplot(133)
plt.plot(Time[start_time:], DataObservedRim[keys_r[2]][start_time:],'r',label='Observed')
plt.plot(TimeSim1, DataSim1Rim[keys_r[2]],'k')
plt.plot(TimeSim2, DataSim2Rim[keys_r[2]],'k',label='Simulated')
put_axis(keys_r[2][-4:],'Rim')


plt.tight_layout()
if save == True:
    plt.savefig(outfile+'vcw-rim%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
#Polygon Rim
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()
print DataObservedRim.keys()
plt.subplot(331)
plt.plot(Time[start_time:], DataObservedRim['temperature-5cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-5cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-5cm'],'k')
put_axis('5cm')

plt.subplot(332)
plt.plot(Time[start_time:], DataObservedRim['temperature-10cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-10cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-10cm'],'k')
put_axis('10cm')

plt.subplot(333)
plt.plot(Time[start_time:], DataObservedRim['temperature-20cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-20cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-20cm'],'k')
put_axis('20cm')
#plt.text(8.,280, 'ignore, observation at wrong location', fontsize=5)


plt.subplot(334)
plt.plot(Time[start_time:], DataObservedRim['temperature-40cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-40cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-40cm'],'k')
put_axis('40cm')

plt.subplot(335)
plt.plot(Time[start_time:], DataObservedRim['temperature-60cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-60cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-60cm'],'k')
put_axis('60cm')

plt.subplot(336)
plt.plot(Time[start_time:], DataObservedRim['temperature-80cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-80cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-80cm'],'k')
put_axis('80cm')

plt.subplot(337)
plt.plot(Time[start_time:], DataObservedRim['temperature-100cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-100cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-100cm'],'k')
put_axis('100cm')

plt.subplot(338)
plt.plot(Time[start_time:], DataObservedRim['temperature-125cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-125cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-125cm'],'k')
put_axis('125cm')

plt.subplot(339)
plt.plot(Time[start_time:], DataObservedRim['temperature-150cm'][start_time:],'r')
plt.plot(TimeSim, DataSimRim['temperature-150cm'],'k')
plt.plot(TimeSim11, DataSim11Rim['temperature-150cm'],'k')
put_axis('150cm')

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-Rim%s-%s.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
#simulation = 'CrunB12R'
simulation = 'CrunB42R12'
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/surface-water_level-C37.dat"%simulation
Data37 = rd.ReadSingleFile(infile)
Data37['time'] = [d/86400. for d in Data37['time']] 
Data37['data'] = [d - 4.83541*0 for d in Data37['data']]
print (infile)
#infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/CrunB3R/surface-water_level-C37-ngbr.dat"
#Data37N = rd.ReadSingleFile(infile)
#Data37N['time'] = [d/86400. for d in Data37N['time']] 
#Data37N['data'] = [d - 4.83541 for d in Data37N['data']]

infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/surface-water_level-C40.dat"%simulation
Data40 = rd.ReadSingleFile(infile)
Data40['time'] = [d/86400. for d in Data40['time']] 
Data40['data'] = [d - 4.87831*0 for d in Data40['data']]

In [ ]:
import plot_water_table2D
times, wt, wt_ss, wt_bottom,thaw_depth = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation)

In [ ]:
lower_point = 4.8284937 
lower_point=  4.80342948 # new mesh
lower_point = 4.81031227 # new smooth mesh

In [ ]:
cellid = 36
cellid = 44 # new mesh
metdata = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-precipitation_rain', cellid=cellid)
surftemp = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-temperature', cellid=cellid)
#plt.plot(metdata['surface-precipitation_rain'])
#plt.show()

In [ ]:
rain = [d *100*900./0.02500 for d in metdata['surface-precipitation_rain']]
E1 = 4.72  + 0.075 * np.exp(rain)
#plt.plot(Data37['time'], Data37['data'], 'k', label='Simulated')
#plt.plot(DataC37['time'], DataC37['WL'],'r', label='Observed (C37)')

#plt.plot(Data37['time'],E1,'*')
#plt.show()
#simulation1 = 'CrunB30R79'
#times1, wt1, wt_ss1, wt_bottom1,thaw_depth1 = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation1)

In [ ]:
Data40 =dict()
time = [t*365.25 for t in times]
#time = [t for t in times]
Data40['time'] = time
Data40['wt'] = wt
Data40['wt_ss'] = wt_ss
Data40['wt_bottom'] = wt_bottom
print (times[0:3])
        
D1 = []
T1 = []
for i,d in enumerate(Data40['wt']):
    if d != 0:
        D1.append(d + lower_point)
        T1.append(Data40['time'][i])
        
D2 = []
T2 = []
for i,d in enumerate(Data40['wt']):
    D2.append(d + lower_point)
    T2.append(Data40['time'][i])
    
#water table bottom
D3 = []
T3 = []
for i,d in enumerate(Data40['wt_bottom']):
    D3.append(d + lower_point)
    T3.append(Data40['time'][i])
TD = []
for d in thaw_depth:
    TD.append(d + lower_point)   

"""
# Second simulation
Data401 =dict()
time1 = [t*365.25 for t in times]
Data401['time'] = time1
Data401['wt'] = wt1
Data401['wt_ss'] = wt_ss1
Data401['wt_bottom'] = wt_bottom1
        
D11 = []
T11 = []
for i,d in enumerate(Data401['wt']):
    if d != 0:
        D11.append(d + lower_point)
        T11.append(Data401['time'][i])
        
    
#water table bottom
D31 = []
T31 = []
for i,d in enumerate(Data401['wt_bottom']):
    D31.append(d + lower_point)
    T31.append(Data401['time'][i])
TD1 = []
for d in thaw_depth1:
    TD1.append(d + lower_point) 
"""

In [ ]:
outpath = '/Users/ajc/AllPostProcessData/BarrowTransect/Aug13-NewTC/'
fig, axs = plt.subplots(1,2,figsize=(10,4))
plt.subplot(121)
plt.plot(Data37['time'], Data37['data'], 'k', label='Simulated')
plt.plot(DataC37['time'], DataC37['WL'],'r', label='Observed (C37)')
plt.axhline(y=4.72)
#plt.plot(Data37['time'], Data37N['data'], 'm')
plt.xlabel('Day')
plt.ylabel('Water level')
plt.ylim((4.4,5.0))
plt.legend()


print (len(DataC39['time']), len(DataC39['WL']))
ax1 = plt.subplot(122)

plt.plot(Data40['time'], TD,'b', label='Thaw depth')
#plt.plot(Data401['time'], TD1,'c', label='Thaw depth (uf TC)') # 2nd simulation
plt.plot(DataC40['time'], DataC40['WL'],'r', label='Observed (C40)')
plt.plot(DataC39['time'], DataC39['WL'],'m', label='Observed (C39)')
plt.axhline(y=lower_point)
plt.plot(T1, D1,'k.',markersize='2.5',label='Simulated')
#plt.plot(T11, D11,'c.',markersize='2.5',label='Simulated (uf TC)') #2nd simulation


#plt.plot(T3, D3,'c',markersize='2.5',label='WT bottom')

#plt.plot(DataC41['time'], DataC41['WL'],'g', label='Observed (C41)')
#plt.plot(DataC42['time'], DataC42['WL'],'c', label='Observed (C42)')
#plt.plot(DataC43['time'], DataC43['WL'],'k', label='Observed (C43)')



plt.xlabel('Day')
plt.ylabel('Water level')
plt.legend(loc='lower left',fontsize=6)
plt.ylim((4.4,5.0))

ax2 = ax1.twinx()
rain = [d*1.e7 for d in metdata['surface-precipitation_rain']]
#temp = [d*1.e0 for d in surftemp['surface-temperature']]
ax2.plot(Data40['time'],rain,'y')
ax2.set_ylabel('Precipitation Rain [m /s] x 1.0e-7',color='y')
ax2.tick_params('y', colors='y')
ax2.set_ylim([-10.0, 2])


plt.savefig(outpath+'%s-transectB-C40-C39.pdf'%simulation, dpi=200)
#plt.savefig(outpath+'%s-transectB-C40-C39-TC-comp.pdf'%simulation, dpi=200)
plt.show()



In [ ]:
# Area B
fig,axs = plt.subplots(1,2, figsize=(14,6))
#fig.subplots_adjust(hsapce=0.2,wspace=0.2)
plt.subplot(121)
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaB/Brun2R/surface-water_level-B64.dat"
Data64 = rd.ReadSingleFile(infile)
z_sim = 5.13042
Data64['time'] = [d/86400. for d in Data64['time']]
Data64['data'] = [(d - z_sim)*1 for d in Data64['data']]

ex = pd.ExcelFile('/Users/ajc/Desktop/ATS/barrow-polygon-data/water_table/NGEE_wells/RevisedNEAR-FINALWaterLevels-NGEESiteB-AC 01_28_2015.xlsx')
#print (ex.sheet_names)
DataB64EX = ex.parse(sheet_name='B64', skiprows=4)
print (DataB64EX.keys())
#DataB64EX = np.array(DataB64EX['Water Level 2013'])
#print (DataB64)
DataB64 = dict()
r = len(DataB64EX['Water Level 2013'])
DataB64['water_level'] = [(d - 4.99)*1 for d in DataB64EX[u'Water Level 2013']]
DataB64['time'] = [(t-time_origin).total_seconds()/86400. for t in DataB64EX['Date2013']]

l= len(DataB64)
plt.plot(Data64['time'], Data64['data'], 'k', label='Simulated')
#print (len(DataB64), len(Data['data'][:l]))
plt.plot(DataB64['time'], DataB64['water_level'],'r', label='Observed B64')
plt.xlabel('Days')
plt.ylabel('Water Depth [m]')
plt.legend()

# B67

plt.subplot(122)
infile = "/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaB/Brun2R/surface-water_level-B67.dat"
Data67 = rd.ReadSingleFile(infile)
z_sim = 5.50947
Data67['time'] = [d/86400. for d in Data67['time']]
Data67['data'] = [(d - z_sim)*1 for d in Data67['data']]

ex = pd.ExcelFile('/Users/ajc/Desktop/ATS/barrow-polygon-data/water_table/NGEE_wells/RevisedNEAR-FINALWaterLevels-NGEESiteB-AC 01_28_2015.xlsx')
print (ex.sheet_names)
DataB67EX = ex.parse(sheet_name='B67', skiprows=4)

#DataB67['water_level'] = np.array(DataB67EX['Water Level 2013'])
DataB67= dict()
DataB67['water_level'] = [(d - 5.44)*1 for d in DataB67EX[u'Water Level 2013']]
DataB67['time'] = [(t - time_origin).total_seconds()/86400. for t in DataB67EX['Date2013']]
print (DataB67['water_level'][:5], DataB67['time'][:5])
plt.plot(Data67['time'], Data67['data'], 'k', label='Simulated')
plt.plot(DataB67['time'], DataB67['water_level'],'r', label='Observed B67')
plt.xlabel('Days')
plt.ylabel('Water Depth [m]')
#plt.legend()

#plt.savefig('polygonAreaB-B64.pdf', dpi=200)
plt.show()
"""
for i in range(len(DataB67['water_level'])):
    if DataB67['water_level'][i] is not 'nan':
        print (i, DataB67['time'][i], DataB67['water_level'][i])
"""